In [7]:
from dataset import PQDDataset
import torch
from torch.utils.data import DataLoader
from dataset import PQDDataset
from torch.optim import Adam
import torch.nn as nn
import torch.fft as fft
from tqdm.auto import tqdm, trange
from sklearn.metrics import f1_score, balanced_accuracy_score
import numpy as np
import pandas as pd
import time
import statistics

In [8]:
device='cpu'

In [10]:
test_df = pd.read_csv('test.csv')
test_dataset = PQDDataset(
                          df=test_df,
                          window_size=32,
                          target_mode=True
                        )

model = torch.load("cnnt_npu_model_seed42_ep6_tl0.8657.pt", map_location=device)
model.to(device)
model.eval()
print(model)

# speeds = []
times = []
dataloader = DataLoader(test_dataset, batch_size=1)
with torch.no_grad():
    with tqdm(dataloader, unit=" batch", mininterval=3) as t:
        for i, (batch, _) in enumerate(t):

            batch = batch.to(device)
            batch = batch.permute(0, 2, 1)
            batch = batch.unsqueeze(2)

            start_time = time.time()
            output = model(batch)
            times.append(time.time() - start_time)

    # end_time = time.time()
    # print(f"Time: {end_time - start_time}")
    # speeds.append(len(test_dataloader) / (end_time - start_time))
    # print(f"Inference speed is {speeds[-1]} frames per second")

# print(f"Average speed is {np.mean(speeds)} frames per second")
print(
    f"Time of inference is min={min(times[10:])*1000} ms, avg={np.median(times[10:])*1000} ms, max={max(times[10:])*1000} ms per frame"
)

Creating window slices:   0%|          | 0/13 [00:00<?, ?it/s]

CONV_NPU(
  (conv): Sequential(
    (0): Conv2d(5, 40, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1), padding_mode=circular)
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(40, 80, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1), padding_mode=circular)
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=(1, 2), stride=(1, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (enc_fc): Linear(in_features=640, out_features=80, bias=True)
  (out_fc): Linear(in_features=80, out_features=3, bias=True)
)


  0%|          | 0/17310 [00:00<?, ? batch/s]

Time of inference is min=0.2346038818359375 ms, avg=0.24938583374023438 ms, max=3.8683414459228516 ms per frame


In [4]:
statistics.median(times) * 1000

0.2448558807373047

In [5]:
statistics.quantiles(times,n=100)[-1] * 1000

0.3120899200439453

In [6]:
0.57+0.53+0.51+0.47

2.08

In [30]:
2.08/ 4

0.52